In [30]:
from bs4 import BeautifulSoup as bs
import time
import sqlite3 
import re
from datetime import datetime
from selenium import webdriver
import json
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os

# Check updates for specific Firms

In [6]:
dictio = {}
dictio['ПАО Сбербанк'] = 3043
name = dictio.keys()





def search_by_companies(*companies):
    for company in companies:
#         create the dict to control the number of new events
        company_pre = {company:0}
        request = urlopen('https://www.e-disclosure.ru/Event/Page?companyId=' + str(dictio[company]) +'&year=2020')
        soup = bs(request, 'lxml')

        
        # It is supposed to infinitely parse ---> don't need to check more than 20 first values
        result = soup.find_all('td')[:15]

        if len(result) > company_pre[company]:
            i = 1
            for td in range ((len(result) - result_pre)//3):

                    print(name, 'Дата публикации - ' + str(result[i].text),result[i+1].text,result[i+1].a.get('href'),sep = '\n')
                    i+=3
            company_pre[company] = len(result)

search_by_companies('ПАО Сбербанк')

NameError: name 'result_pre' is not defined

# SQL database

# Test and training

In [ ]:
conn = sqlite3.connect('Chinook_Sqlite.sqlite')

# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor = conn.cursor()

cursor.execute("SELECT Name FROM Artist LIMIT 3")

# Получаем результат сделанного запроса
results = cursor.fetchall()
results2 =  cursor.fetchall()

print(*results)   # [('A Cor Do Som',), ('Aaron Copland & London Symphony Orchestra',), ('Aaron Goldberg',)]

# Не забываем закрыть соединение с базой данных
conn.close()

# Working 

In [ ]:
conn = sqlite3.connect('Interfax_bot.sqlite')

cursor = conn.cursor()


cursor.execute("select Client_name from Clients")
print(cursor.fetchall())

### Adding new clients

### Check new or existing clients

#### Add client function

In [ ]:
# client_chat = 779224126
# client_companies = '[2314,4526,124]'

def add_client(client_chat,client_companies):
    """ Adding new client to the database"""

    conn = sqlite3.connect('Interfax_bot.sqlite')

    cursor = conn.cursor()

    # Данные нужно передавать кортежем. Даже одну запись
    
    
    cursor.execute("insert into Clients values (Null, ?, ?);", (client_chat, client_companies))

    conn.commit()

#### Add or update client function(The main function). Don't need to use the previous function, since it is included in the current one

In [ ]:
def add_update(name,companies):    
    
    conn = sqlite3.connect('Interfax_bot.sqlite')

    cursor = conn.cursor()

    cursor.execute('select ID from Clients where Client_name = ?;',(name,))

    res = cursor.fetchone()

    if res == None:
        add_client(name,companies)
    else:
        cursor.execute('''update Clients 
                          set Client_name = ?, companies = ? 
                          where ID = ?;''',(name, companies, res[0]))
        conn.commit() 
    # res[0] is the ID value itself, since fetchone returns a tuple
       

In [ ]:
# add_update(987,'[12345,6789]')
# add_update(1345,'[1234]')

#### Function to retrieve company codes as a set

In [38]:
def retrieve_code(string):
    dic = list(map(str, string.replace('{','').replace('}','').split(',')))
    dic = set(dic)
    return dic

In [ ]:
print(retrieve_code('{3445,4252,2156,1345}'))

#### Function to create a string from a set

In [39]:
def create_string(array):
    
    string = '{'
    for i in array:
        if i == '':
            continue
        string = string + str(i) + ','
    string = string[:-1] + '}'
    return string

In [ ]:
i = set([5434, 134, 670, 243])

print(create_string(i))

In [ ]:
clients = {779224126 : []} # a new client's chat_id is added ones he presses /start

diction ={'18278':['Ale','Ma','Joh'], '2464':['a','b','c']} # the reverse dictio containing company numbers and clients' chat_id


# Algorithm to add clients code to every company

In [43]:
# This needs to be done at the moment the companies' set is created in the python/
# At that moment I need to reverse it and add the client_chat to every company code in the other table
# I will need to add the mechanism of checking the existing and -non-existing company codes later


# client = 1234
# companies = '{1234,567,945}'

def add_subscribers(companies,client):
    """ Updates the set of subscribers for every company the new client has chosen"""
    

    conn = sqlite3.connect('Company_codes_to_clients.sqlite')
    cursor = conn.cursor()
    
#   Since this module will work right after the client chooses companies, the given array of companies codes will be a set
    for i in companies:

    # retrieve the existing string of clients for the company
    # convert it to set
    # add the client to this set
    # convert back to string
    # update the table

        cursor.execute('select Chat_codes from Companies where Company_name = ?;',(i,))
        chat_codes = retrieve_code((cursor.fetchone()[0]))
        chat_codes.add(client)
        update = create_string(chat_codes)
        cursor.execute("""
                        update Companies 
                        set Chat_codes = ?
                        where Company_name = ?

                        """,(update,i))
        conn.commit()


In [ ]:
conn = sqlite3.connect('Interfax_bot.sqlite')

In [ ]:
add_subscribers({567, 945},5876413)

In [ ]:
cursor.execute('select Chat_codes from Companies where ID = 4')
print(retrieve_code(cursor.fetchone()[0]))

# Algorithm to search and add companies(for the client)

### Эту хрень нужно поставить внутрь хендлеров
### Какая-то часть работает - это радует

In [7]:
def choose_companies(message):
    conn = sqlite3.connect('Company_codes_to_clients.sqlite')
    cursor = conn.cursor()
    cursor.execute('select Company_name from Companies')
    companies = cursor.fetchall()
    
    comp_list = {}
    
    bot.send_message(message.chat.id, 'Напиши название компании.')
    company = message.text
    for i in companies:
        if company in i[0]:
            comp_list.add(i[0])
            bot.send_message(message.chat.id, 'Добавил')

        
    bot.send_message(message.chat.id, 'Напиши имя следующей компании если хочешь добавить еще. Когда закончишь, напиши "/end"')
    
    return comp_list
# for i in dictio.keys():
#     if name.lower() in i: 
#         print(i)

### Adding the company_names and codes to DB - Done

In [ ]:
# I need to read it from Sqlite and not keep in memory, since the users will not add new companies constantly

companies_names = pd.read_csv('interfax_companies.csv', encoding = 'windows-1251', index_col = 0)

companies = companies_names['company'].to_list()
ids = companies_names['comp_id'].to_list()

dictio = {}
companies_low = [i.lower() for i in companies]

for i in range(len(companies)):
    dictio[companies_low[i]] = ids[i]
print(dictio)


# UNCOMMENT THIS WHEN NEED TO UPDATE THE DATABASE - BUT BE CAREFUL. IT WILL REMOVE ALL THE EXISTING CLIENTS FOR EACH COMPANY
# SO CREATE THE DICTION WITH ONLY NEW COMPANIES AND DO THAT

# conn = sqlite3.connect('Company_codes_to_clients.sqlite')

# cursor = conn.cursor()


# for i in range(len(companies)):
#     cursor.execute('insert into Companies values (Null, ?,?,?)',(companies_low[i],ids[i],'{}'))
# conn.commit()

# Telegram bot

### News check

In [ ]:
def news_check(message):
    url = 'https://www.e-disclosure.ru/'
# set up driver for working with js - BS4 will not find the table 
    driver = webdriver.Firefox()
    driver.implicitly_wait(30)
    driver.get(url)
# press button 'search'
    python_button = driver.find_element_by_id('butt') #FHSU
    python_button.click() #click fhsu link
    
    soup = bs(driver.page_source, 'lxml')
    res = soup.find_all('a')

    for a in range(42,72,2):
        bot.send_message(message.chat.id, res[a].text + '\n' + str(res[a+1].text) + ' ' + str(res[a+1].get('href')))


### Subscription module

#### Finding table with last 20 news 

In [34]:
def find_news():
    url = 'https://www.e-disclosure.ru/poisk-po-soobshheniyam'

    driver = webdriver.Firefox()
    driver.implicitly_wait(30)
    driver.get(url)

    python_button = driver.find_element_by_id('butt') #FHSU
    python_button.click() #click fhsu link
    #  Выбрать пункт 20 и нажать
    python_button2 = driver.find_element_by_xpath('//*[@id="pageSize"]/option[2]')
    python_button2.click()

    time.sleep(4)
    soup = bs(driver.page_source, 'lxml')
    res = soup.find_all('tbody')

    global table
    table = res[-9]

    driver.quit()


### Retrieving code

In [35]:
def firm_code_from_link(link):
    equal = link.rfind('=')
    code = link[equal+1:]
    return code

### Sending news to all the firms subscribers

### Now works with DB

In [96]:
def subscribers(link,info):
    conn = sqlite3.connect('Company_codes_to_clients.sqlite')
    
    cursor = conn.cursor()
    code = firm_code_from_link(link)
    
    cursor.execute('select Chat_codes from Companies where Code = ?', (code,))
    chat_codes = cursor.fetchall()
    
    
    if len(chat_codes) == 0:
        print(0)
    else:
        for i in retrieve_code(chat_codes[0][0]):
            bot.send_message(i,info)
        
   

In [97]:
subscribers('https://www.e-disclosure.ru/portal/company.aspx?id=65','123')

0


In [99]:
subscribers('https://www.e-disclosure.ru/portal/company.aspx?id=65','hello')

0


#### Choosing only new events with the help of date-time and returning them

In [104]:
td_last = 270520201800
def send_news():

    global td_last
    
    #  Here i need to define all the variables I will use in this module
    def number_date(text):
        '''Function to convert date into an int for comparison and searching relevant news'''
        pre_res = text.split()
        one = pre_res[0].replace('.','') 
        two = pre_res[1].replace(':','')
        res = int(one+two)
        return res



    iterat = table.contents  # all tr in the table

    
        
    for child in range(0,len(iterat),2):
    # for some reason there are empty cells in [0],[2],[4] and etc. rows, so I need to go through odd ones
        date = iterat[child].contents[1].text # first td with the date and time
        if number_date(date) > td_last:
    #         iterat[child].contents[3] is the second td containing company name and the event name. However, there are two <a> tags, 
    #         one with company name, the other with the event. Therefore another method, contents[0].text - company and contents[3].text - event, is used.
            info = str(iterat[child].contents[3].contents[0].text) + '\n' + str(iterat[child].contents[3].contents[3].text) + ' ' + str(iterat[child].contents[3].contents[3].get('href'))
            link = iterat[child].contents[3].contents[0].get('href') 


            subscribers(link, info)

        # td_last is the last time we checked news - it is equiualent to the top row news data in the current session.



    td_last = number_date(iterat[0].contents[1].text) 




0
0
0
0
0
0
0


ApiException: A request to the Telegram API was unsuccessful. The server returned HTTP 400 Bad Request. Response body:
[b'{"ok":false,"error_code":400,"description":"Bad Request: chat_id is empty"}']

### The Bot

In [105]:
import telebot

token = '1145245715:AAGmd4yheC8AlBpZ5ssHqQm9-1hpt_mkAT0'

bot = telebot.TeleBot(token)

keyboard1 = telebot.types.ReplyKeyboardMarkup()
keyboard1.row('Последние новости', 'Выбрать компании')

keyboard2 = telebot.types.InlineKeyboardMarkup(); #наша клавиатура
key_one = telebot.types.InlineKeyboardButton(text='Новости', callback_data='news'); #кнопка «Да»
keyboard2.add(key_one); #добавляем кнопку в клавиатуру
key_two = telebot.types.InlineKeyboardButton(text='Выбрать компании', callback_data='companies');
keyboard2.add(key_two);

comp_list = set()

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет, я бот раскрытия. В текущей версии я буду отправлять тебе новости о последних раскрытиях в реальном времени.')
    bot.send_message(message.chat.id, 'Какую опцию выберешь?', reply_markup = keyboard2)
    chat_id = message.chat.id
#     I need to add clients chat_id to SQLite database later
    
@bot.message_handler(content_types = ['text'])
def init_choose_companies(message):
    conn = sqlite3.connect('Company_codes_to_clients.sqlite')
    cursor = conn.cursor()
    cursor.execute('select Company_name from Companies')
    
    comp = cursor.fetchall()
    
    global companies
    
    companies = list()
    
    for i in comp:
        companies.append(i[0]) 
    
    

    msg = bot.send_message(message.chat.id, 'Напиши название компании')
    bot.register_next_step_handler(msg,choose_companies)
    
    
def  choose_companies(message):
    
    global comp_list
    
    company = message.text
    
    if company =='/end':
        
#         for i in comp_list:
#             bot.send_message(message.chat.id, str(i))
# add all the chosen companies to the DB
        add_subscribers(comp_list, message.chat.id)
        
        
        bot.send_message(message.chat.id,'Thanks')
#         here I nullify the variable comp_list for the next user - need to check whether this works
        comp_list = set()
        
    else:
        comp_results = list()
        for i in companies:
            if company.lower() in i:
                comp_results.append(i)
        
        if len(comp_results) == 1:
            bot.send_message(message.chat.id, 'Добавил')
            comp_list.add(comp_results[0])
            msg = bot.send_message(message.chat.id, 'Напиши название следующей компании или "/end" если хочешь закончить.')
            bot.register_next_step_handler(msg, choose_companies)
        
        elif len(comp_results) > 1:
            msg = bot.send_message(message.chat.id, 'Я нашел несколько результатов. Напиши название того, который нужен')
            
            for a in comp_results:
                bot.send_message(message.chat.id, str(a))
            
            bot.register_next_step_handler(msg, choose_companies)
        
        else:
            msg = bot.send_message(message.chat.id, 'Не смог найти такую компанию.' +'\n' + 'Напиши название следующей компании или "/end" если хочешь закончить.')
            bot.register_next_step_handler(msg, choose_companies)    
    
#         if company not in companies:
#             bot.send_message(message.chat.id, 'Не смог найти такую компанию')
#             msg = bot.send_message(message.chat.id, 'Напиши название следующей компании или "/end" если хочешь закончить.')
#             bot.register_next_step_handler(msg,choose_companies)

    
    
    
        
    

    
    
# for i in dictio.keys():
#     if name.lower() in i: 
#         print(i)    
        

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    chat = call.message
    if call.data == "news": #call.data это callback_data, которую мы указали при объявлении кнопки
        find_news()
        send_news()
    elif call.data == "companies":
        init_choose_companies(chat)
        
        
        

bot.polling()

0
0
0


2020-05-28 22:16:55,407 (util.py:68 WorkerThread14) ERROR - TeleBot: "ApiException occurred, args=('A request to the Telegram API was unsuccessful. The server returned HTTP 400 Bad Request. Response body:\n[b\'{"ok":false,"error_code":400,"description":"Bad Request: chat_id is empty"}\']',)
Traceback (most recent call last):
  File "C:\Users\azhur\Anaconda3\lib\site-packages\telebot\util.py", line 62, in run
    task(*args, **kwargs)
  File "<ipython-input-105-9a0892d4d8d7>", line 111, in callback_worker
    send_news()
  File "<ipython-input-104-464f6a00d9bc>", line 31, in send_news
    subscribers(link, info)
  File "<ipython-input-96-d8def814398e>", line 15, in subscribers
    bot.send_message(i,info)
  File "C:\Users\azhur\Anaconda3\lib\site-packages\telebot\__init__.py", line 625, in send_message
    reply_markup, parse_mode, disable_notification, timeout))
  File "C:\Users\azhur\Anaconda3\lib\site-packages\telebot\apihelper.py", line 159, in send_message
    return _make_request(

0
0
0
0


In [5]:
print(companies)

['ао "дом.рф"', 'пао банк зенит', 'пао "калужская сбытовая компания"', 'пао мособлбанк', 'оао "левенгук"', 'пао "гк "нижегородский"', 'пао "кзф"', 'ао бпжт', 'оао "швейная фабрика "победа"', 'ао "пз "расцвет"', 'оао "мповти"', 'ао "авангард-агро"', 'ооо «сфо сфи»', 'оао "демидов-авто"', 'ао "ифз"', 'оао "дмз"', 'ао "промтрактор"', 'ао "торговый дом казанский цум"', 'оао "соз"', 'ао "бываловский машиностроительный завод"', 'пао "воэ" пао "волгоградоблэлектро"', 'ооо "лента"', 'оао "рорз"', 'ао  "мехколонна"', 'ао «смп банк»', 'ао "мособлстрой № 5"', 'аор "туринский цбз"', 'псф "автодизель-сервис" ао', 'ооо "каркаде"', 'ао "софтлайн трейд"', 'ао «сбербанк управление активами»', 'ао "элион"', 'зао «иа «московский»', 'оао "цнииб"', 'ао "газпром газораспределение сыктывкар"', 'оао "лакша"', 'ао "ата"', 'банк "вятич" (пао)', 'банк "вбрр" (ао)', 'пао «уптк «чебоксарское»', 'оао "агп"', 'пао "центрторг"', 'пао  банк "сиаб"', 'оао смп', 'ао «ладья-финанс»', 'ао "атомэнергопром"', 'пао "каменско

In [4]:
print(diction)

NameError: name 'diction' is not defined

# Finished Modules

## Companies_List

In [ ]:
url = 'https://www.e-disclosure.ru/poisk-po-soobshheniyam'

driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.get(url)

python_button = driver.find_element_by_id('butt') #FHSU
python_button.click() #click fhsu link

python_button2 = driver.find_element_by_id('pageSize')
python_button2.click()

soup = bs(driver.page_source, 'lxml')
res = soup.find_all('a')
# select all the relevant passages from the table. Empirically found them.
companies = {}
for a  in range(46,2445,2):
#     remove comma only when need to create a dict
#     companies[res[a].text] = res[a].get('href')[-5:]
    print(res[a].text, res[a].get('href'))

driver.quit()

for i in companies.keys():
    if companies[i].startswith('='):
        companies[i] = companies[i].replace('=','')

    if companies[i].startswith('d='):
        companies[i] = companies[i].replace('d=','')
companies

In [ ]:
company = list(companies.keys())
comp_id = list(companies.values())

data = {'company':company, 'comp_id':comp_id}
df = pd.DataFrame.from_dict(data)
df

In [ ]:
# df.to_csv('interfax_companies.csv', encoding = 'windows - 1251')